# Project Luna 16

In this project, you will develop a deep learning model to classify lung nodules as benign or malignant from 3D CT scans, utilizing the LUNA16 dataset. This task involves data preprocessing, model design, training, and evaluation, offering hands-on experience with medical image analysis and deep learning in PyTorch.

## 1. Create Annotation Data
As the first step, we will need to load the annotation data from Kaggle's data page: [Luna 16 Lung Cancer Dataset on Kaggle](https://www.kaggle.com/datasets/fanbyprinciple/luna-lung-cancer-dataset)

1.1 Download the annotation dataset from Kaggle.

In [ ]:
# Upload kaggle.json before running this cell
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d 'fanbyprinciple/luna-lung-cancer-dataset'
! unzip -q luna-lung-cancer-dataset.zip -d luna16

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/fanbyprinciple/luna-lung-cancer-dataset
License(s): CC-BY-SA-3.0
100% 330M/330M [00:10<00:00, 39.2MB/s]
100% 330M/330M [00:10<00:00, 32.2MB/s]


In [ ]:
! pip install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 9.8 MB/s eta 0:00:00


1.2 Load the `candidates_V2.csv` file as a data frame. Display the first 5 rows.

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import SimpleITK as sitk


candidates = pd.read_csv('luna16/candidates_V2/candidates_V2.csv')
annotations = pd.read_csv("luna16/annotations.csv")


candidates.head()



,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0


1.3 Display the number of class 0 (benign) records and the number of class 1 (malignant) records. Your results should indicate that the two classes are highly imbalanced.

In [ ]:
class_counts = candidates['class'].value_counts()


#class 0 and 1
print("Number of Class 0 (Benign) Records:", class_counts[0])
print("Number of Class 1 (Malignant) Records:", class_counts[1])

#imbalance
print("\nClass Distribution:\n", class_counts)


Number of Class 0 (Benign) Records: 753418
Number of Class 1 (Malignant) Records: 1557

Class Distribution:
 class
0    753418
1      1557
Name: count, dtype: int64


## 2. Find Nodule Locations
In the annotation dataset, the center of each identified lung nodule is marked with its 3D coordinates. We need to convert these coordinates into three indices to identify the specific subarray in each CT scan tensor that corresponds to the nodule.

Please follow the steps outlined in the LUNA16DataPreparation notebook to generate a CSV file named `candidates_processed.csv`, which will store the indices for the center of each lung nodule.

2.1 Load the `subset0.zip` from Google Drive using the file ID '1OFa8UhDvCrcTj1VkFLa7RjifEqMD4TAa'. Extract the zip file to reveal the .mhd and .raw files.

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id='1OFa8UhDvCrcTj1VkFLa7RjifEqMD4TAa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('subset0.zip')

! unzip -q subset0.zip

2.2 Create a Pandas data frame that contains these 8 colums: `seriesuid`, `coordX`, `coordY`, `coordZ`, `class`, `index`, `row`, `col`. The last three columns should be calculated using the three coordiantes and the information about the origin and spacing of the corresponding CT scan.

In [ ]:
all_mhd = glob.glob('subset0/*.mhd')
name = all_mhd[0]
start_index = name.find('1.')
end_index = name.find('.mhd')

print("ID:", name[start_index:end_index])
ids = [name[start_index:end_index] for name in all_mhd]
print("ID list:", ids)

samples = candidates[(candidates['class'] == 1) &
           (candidates['seriesuid'].isin(ids))].sample(5)

num_rows = len(ids)
ct_info = pd.DataFrame(np.zeros([num_rows, 6]), columns=['originX', 'originY', 'originZ', 'spacingX', 'spacingY', 'spacingZ'], index=ids)
ct_info.head()

ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.294188507421106424248264912111
ID list: ['1.3.6.1.4.1.14519.5.2.1.6279.6001.294188507421106424248264912111', '1.3.6.1.4.1.14519.5.2.1.6279.6001.333145094436144085379032922488', '1.3.6.1.4.1.14519.5.2.1.6279.6001.323859712968543712594665815359', '1.3.6.1.4.1.14519.5.2.1.6279.6001.134996872583497382954024478441', '1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059', '1.3.6.1.4.1.14519.5.2.1.6279.6001.313835996725364342034830119490', '1.3.6.1.4.1.14519.5.2.1.6279.6001.141069661700670042960678408762', '1.3.6.1.4.1.14519.5.2.1.6279.6001.137763212752154081977261297097', '1.3.6.1.4.1.14519.5.2.1.6279.6001.295298571102631191572192562523', '1.3.6.1.4.1.14519.5.2.1.6279.6001.566816709786169715745131047975', '1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674661221381920536987', '1.3.6.1.4.1.14519.5.2.1.6279.6001.249530219848512542668813996730', '1.3.6.1.4.1.14519.5.2.1.6279.6001.979083010707182900091062408058', '1.3.6.1.4.1.14519.5.2.1.6279.6001.10

,originX,originY,originZ,spacingX,spacingY,spacingZ
1.3.6.1.4.1.14519.5.2.1.6279.6001.294188507421106424248264912111,0.0,0.0,0.0,0.0,0.0,0.0
1.3.6.1.4.1.14519.5.2.1.6279.6001.333145094436144085379032922488,0.0,0.0,0.0,0.0,0.0,0.0
1.3.6.1.4.1.14519.5.2.1.6279.6001.323859712968543712594665815359,0.0,0.0,0.0,0.0,0.0,0.0
1.3.6.1.4.1.14519.5.2.1.6279.6001.134996872583497382954024478441,0.0,0.0,0.0,0.0,0.0,0.0
1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for id in ids:
    mhd_path = 'subset0/{}.mhd'.format(id)
    ct_mhd = sitk.ReadImage(mhd_path)
    ct_info.loc[id, ['originX', 'originY', 'originZ']] = list(ct_mhd.GetOrigin())
    ct_info.loc[id, ['spacingX', 'spacingY', 'spacingZ']] = list(ct_mhd.GetSpacing())
    if ct_mhd.GetDirection() != (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0):
        print(id, ct_mhd.GetDirection())
print(ct_info.shape)
ct_info.head()

1.3.6.1.4.1.14519.5.2.1.6279.6001.128023902651233986592378348912 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.9999999999999999)
1.3.6.1.4.1.14519.5.2.1.6279.6001.564534197011295112247542153557 (0.9999999999999999, 0.0, 0.0, 0.0, 0.9999999999999999, 0.0, 0.0, 0.0, 1.0)
1.3.6.1.4.1.14519.5.2.1.6279.6001.146429221666426688999739595820 (0.9999999999999999, 0.0, 0.0, 0.0, 0.9999999999999999, 0.0, 0.0, 0.0, 1.0)
1.3.6.1.4.1.14519.5.2.1.6279.6001.227962600322799211676960828223 (0.9999999999999999, 0.0, 0.0, 0.0, 0.9999999999999999, 0.0, 0.0, 0.0, 1.0)
1.3.6.1.4.1.14519.5.2.1.6279.6001.826812708000318290301835871780 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.9999999999999999)
1.3.6.1.4.1.14519.5.2.1.6279.6001.657775098760536289051744981056 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.9999999999999999)
1.3.6.1.4.1.14519.5.2.1.6279.6001.310626494937915759224334597176 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.9999999999999999)
(89, 6)


,originX,originY,originZ,spacingX,spacingY,spacingZ
1.3.6.1.4.1.14519.5.2.1.6279.6001.294188507421106424248264912111,-145.300003,-174.399994,-347.250000,0.585938,0.585938,1.25
1.3.6.1.4.1.14519.5.2.1.6279.6001.333145094436144085379032922488,-182.399994,-187.100006,-382.500000,0.781250,0.781250,2.50
1.3.6.1.4.1.14519.5.2.1.6279.6001.323859712968543712594665815359,-139.500000,-167.600006,-345.345001,0.595703,0.595703,1.25
1.3.6.1.4.1.14519.5.2.1.6279.6001.134996872583497382954024478441,-238.100006,-220.000000,-273.250000,0.859375,0.859375,1.25
1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059,-187.699997,-108.300003,-194.000000,0.548828,0.548828,1.25


In [ ]:
direction_a = np.array([1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]).reshape(3, 3)
print(direction_a)

coordX, coordY, coordZ = samples.loc[samples.index[0], ['coordX', 'coordY', 'coordZ']]
print(coordX, coordY, coordZ)
id = samples.loc[samples.index[0], 'seriesuid']
origin_a = ct_info.loc[id, ['originX', 'originY', 'originZ']].values.reshape([-1])
vxSize_a = ct_info.loc[id, ['spacingX', 'spacingY', 'spacingZ']].values.reshape([-1])
coord_a = np.array([coordX, coordY, coordZ])
cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
cri_a = np.round(cri_a)
index = int(cri_a[2])
row = int(cri_a[1])
col = int(cri_a[0])
index, row, col


mhd_path = 'subset0/{}.mhd'.format(id)
ct_mhd = sitk.ReadImage(mhd_path)
ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
ct_a.clip(-1000, 1000, ct_a)
ct_chunk = ct_a[(index-16):(index+16), (row-24):(row+24), (col-24):(col+24)]
ct_chunk.shape

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
-93.910973 63.824128 -98.809998


(32, 48, 48)

In [ ]:
count = 0

candidates_small = candidates[candidates['seriesuid'].isin(ids)]
print(len(candidates_small))

for idx in candidates_small.index: # nodules supported by the CT scan data

    # Extract the coordinates of the nodule
    coordX, coordY, coordZ = candidates_small.loc[idx, ['coordX', 'coordY', 'coordZ']]

    # Extract the corresponding CT scan ID.
    id = candidates_small.loc[idx, 'seriesuid']

    # Convert the coordiates to indices
    origin_a = ct_info.loc[id, ['originX', 'originY', 'originZ']].values.reshape([-1])
    vxSize_a = ct_info.loc[id, ['spacingX', 'spacingY', 'spacingZ']].values.reshape([-1])
    coord_a = np.array([coordX, coordY, coordZ])
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)

    #indices to the data frame
    candidates_small.loc[idx, 'index'] = int(cri_a[2])
    candidates_small.loc[idx, 'row'] = int(cri_a[1])
    candidates_small.loc[idx, 'col'] = int(cri_a[0])

    count += 1
    if count % 1000 == 0:
        print(count)

79135


<ipython-input-9-80c9d7d077ce>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates_small.loc[idx, 'index'] = int(cri_a[2])
<ipython-input-9-80c9d7d077ce>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates_small.loc[idx, 'row'] = int(cri_a[1])
<ipython-input-9-80c9d7d077ce>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000


2.3 Save the data frame as a CSV file named `candiadates_processed.csv`. This allows you to skip Step 2.2 in future calculations.




In [ ]:
candidates_small.isnull().sum()

# Convert the indices to integers
candidates_small = candidates_small.copy() # remove the connection between candidates and candidates_small
candidates_small[['index', 'row', 'col']] = candidates_small[['index', 'row', 'col']].astype(int)

candidates_small.to_csv('candidates_processed.csv', index=False)

temp = pd.read_csv('candidates_processed.csv')
temp.head()

# Google Drive
from google.colab import drive
drive.mount('/content/drive')

candidates_small.to_csv('drive/MyDrive/candidates_processed.csv', index=False)

ct_info.to_csv("drive/MyDrive/ct_info.csv")
temp = pd.read_csv("drive/MyDrive/ct_info.csv", index_col=0) # Set the first column to be the indices
temp.head()


Mounted at /content/drive


,originX,originY,originZ,spacingX,spacingY,spacingZ
1.3.6.1.4.1.14519.5.2.1.6279.6001.294188507421106424248264912111,-145.300003,-174.399994,-347.250000,0.585938,0.585938,1.25
1.3.6.1.4.1.14519.5.2.1.6279.6001.333145094436144085379032922488,-182.399994,-187.100006,-382.500000,0.781250,0.781250,2.50
1.3.6.1.4.1.14519.5.2.1.6279.6001.323859712968543712594665815359,-139.500000,-167.600006,-345.345001,0.595703,0.595703,1.25
1.3.6.1.4.1.14519.5.2.1.6279.6001.134996872583497382954024478441,-238.100006,-220.000000,-273.250000,0.859375,0.859375,1.25
1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059,-187.699997,-108.300003,-194.000000,0.548828,0.548828,1.25


## 3. Create Data Tensors

The neural network model we will build with PyTorch requires the data to be presented in the form of a torch tensor. The input tensor should be 4-dimensional, with the dimensions representing the nodule index, channel, row, and column, respectively.

3.1 Write a double for-loop to extract the CT scan data for **the first 5,000*** nodules:
- The outer for loop goes through all the distince seriesuid's.
- For each iteration of the outer loop, load the corresponding CT-scan file and create a torch tensor to represent the scan.
- Create an inner-loop that goes through the nodules corresponding to the seriesuid:
    - Load the (index, row, col) tuple of this nodule from the data frame.
    - Extract a 32x48x48 chunk centered at the (index, row, col). If the nodule is near the edge of the image and there is not enough indices to extract, please pad with zeros to keep the overall shape unchanged.
    - Use a 4D tensor to contain all the 32x48x48 chunks. The first dimension of the 4D tensor is the index of nodule.

You may modify the above procedure as you like. Make sure that you are able to obtain a 4D tensor that contains all nodule data. **Display the shape of the 4D tensor.** The shape of the tensor should be (5000, 32, 48, 48).

**Remark** Due to the memory limit, it is impossible to load all nodule images into simultanously. Therefore, the number of nodules required in this section is reduced to 5,000. Feel free to adjust this number to prevent the out-of-memory error.

In [ ]:
import pandas as pd
import numpy as np
import torch
import SimpleITK as sitk
import glob

data = pd.read_csv('candidates_processed.csv')

num_nodules = 5000

#empty input tensor and an empty output tensor
chunks = torch.zeros([num_nodules, 32, 48, 48])
labels = torch.zeros([num_nodules])



3.2 Create a 1D tensor to contain all the class information.

In [ ]:
all_files = glob.glob('subset0/*.mhd')
count = 0

for file in all_files:
    ct_mhd = sitk.ReadImage(file)
    ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
    ct_a = ct_a.clip(-1000, 1000)


    start_index = file.find('1.')
    end_index = file.find('.mhd')
    series_uid = file[start_index:end_index]

    nodules = data[data['seriesuid'].str.contains(series_uid)]

    for nodule_index in nodules.index:

        index, row, col = data.loc[nodule_index, ["index", "row", "col"]]


        ct_chunk = torch.zeros([32, 48, 48])

        # handle edge cases for padding
        index_min, index_max = max(0, index - 16), min(ct_a.shape[0], index + 16)
        row_min, row_max = max(0, row - 24), min(ct_a.shape[1], row + 24)
        col_min, col_max = max(0, col - 24), min(ct_a.shape[2], col + 24)

        chunk = ct_a[index_min:index_max, row_min:row_max, col_min:col_max]
        chunk_shape = chunk.shape


        ct_chunk[:chunk_shape[0], :chunk_shape[1], :chunk_shape[2]] = torch.tensor(chunk)


        chunks[count, :, :, :] = ct_chunk
        labels[count] = data.loc[nodule_index, 'class']

        count += 1
        print(f"Processed nodule {count}/{num_nodules}")

        if count == num_nodules:
            break
    if count == num_nodules:
        break

print(f"Final shape of chunks tensor: {chunks.shape}")
print(f"Final shape of labels tensor: {labels.shape}")

Streaming output truncated to the last 5000 lines.
Processed nodule 3/5000
Processed nodule 4/5000
Processed nodule 5/5000
Processed nodule 6/5000
Processed nodule 7/5000
Processed nodule 8/5000
Processed nodule 9/5000
Processed nodule 10/5000
Processed nodule 11/5000
Processed nodule 12/5000
Processed nodule 13/5000
Processed nodule 14/5000
Processed nodule 15/5000
Processed nodule 16/5000
Processed nodule 17/5000
Processed nodule 18/5000
Processed nodule 19/5000
Processed nodule 20/5000
Processed nodule 21/5000
Processed nodule 22/5000
Processed nodule 23/5000
Processed nodule 24/5000
Processed nodule 25/5000
Processed nodule 26/5000
Processed nodule 27/5000
Processed nodule 28/5000
Processed nodule 29/5000
Processed nodule 30/5000
Processed nodule 31/5000
Processed nodule 32/5000
Processed nodule 33/5000
Processed nodule 34/5000
Processed nodule 35/5000
Processed nodule 36/5000
Processed nodule 37/5000
Processed nodule 38/5000
Processed nodule 39/5000
Processed nodule 40/5000
Proces

3.3 Split the 4D tensor into a training set and a test set. Display their shapes.

In [ ]:
from sklearn.model_selection import train_test_split

train_chunks, test_chunks, train_labels, test_labels = train_test_split(
    chunks.numpy(), labels.numpy(), test_size=0.2, random_state=42)

print(f"Shape of training set (chunks): {train_chunks.shape}")
print(f"Shape of test set (chunks): {test_chunks.shape}")
print(f"Shape of training set (labels): {train_labels.shape}")
print(f"Shape of test set (labels): {test_labels.shape}")

Shape of training set (chunks): (4000, 32, 48, 48)
Shape of test set (chunks): (1000, 32, 48, 48)
Shape of training set (labels): (4000,)
Shape of test set (labels): (1000,)
